# Model Creator

## How to Use
1. After labeling your images in IBM Cloud Annotations, export your file as CreateML.
<img src="https://github.com/SamuelFolledo/Convolutional-Neural-Network-Practice/blob/master/static/exportAsCreateML.png" width="700">

2. Open terminal and change directory to this project
3. For best practices, ensure Python's virtualenv is installed which will help us create an isolated Python environment.

    `pip install virtualenv`


4. Start virtual environment and install packages we need.
    - Create virtual environment
    
        `virtualenv TuriSample`
    
    - Run the environment
    
        `source bin/activate`
    
    - Install TuriCreate and JupyterNotebook
    
        `pip install turicreate`
    
        `pip install jupyter`
    
    
5. Open JupyterNotebook, a Python editor which will open in your browser.

    `jupyter notebook`


6. Unzip the file downloaded from IBM Cloud Annotations and you should see a folder like *project-name-bucket*
7. Rename the folder to **Images**
7. Open the Images folder, look for **annotations.json** and move it outside of the folder. Your folder should look like the image below.
<img src="https://github.com/SamuelFolledo/Convolutional-Neural-Network-Practice/blob/master/static/folderSetUp.png?raw=true" width="700">

In [4]:
import json
import csv
import os

jsonFileName = "annotations.json"

projectDirectory = os.getcwd() #get currrent working directory
imagesDirectory = projectDirectory + "/Images" # directory path where all our labeled images is contained

def main():
    #read json
    with open(jsonFileName) as f:
        imagesDictionary = json.load(f)
    
    #create a csv to write on
    with open('annotations.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, escapechar=' ', quoting=csv.QUOTE_NONE)
        writer.writerow(["path,annotations"])
        for imageDict in imagesDictionary: # start converting each images
            # add (path) first column
            imageName = imageDict["image"]
            imagePath = "{}/{}".format(imagesDirectory, imageName)
            row = imagePath
            
            # START annotation (second column)
            row += ",\"["
            for index, labelDictionary in enumerate(imageDict["annotations"]): # loop through each label in each images
                labelDictionary["type"] = "rectangle" #add shape type
                labelString = json.dumps(labelDictionary) #converts dictionary into string
                if labelDictionary["label"] == "tounge": #handle mislabeled labels
                    labelDictionary["label"] = "tongue"
                    labelString = json.dumps(labelDictionary)
                singleQuotedString = labelString.replace('"', "\'") #turn labelString's double quotes to single quotes
                row += singleQuotedString
                if index != len(imageDict["annotations"]) - 1: #while not the last annotation, add comma
                    row += ","
            row += "]\""
            print(row)
            writer.writerow([row])

if __name__ == "__main__":
    main()

/Users/samuelfolledo/Desktop/Convolutional-Neural-Network-Practice/Images/ac6779a1-d7e1-4f3f-93e7-350ad76ff1ad.jpg,"[{'label': 'tonsil', 'coordinates': {'x': 221, 'y': 508, 'width': 111, 'height': 151}, 'type': 'rectangle'},{'label': 'tonsil', 'coordinates': {'x': 504, 'y': 487, 'width': 184, 'height': 145}, 'type': 'rectangle'},{'label': 'uvula', 'coordinates': {'x': 351, 'y': 480, 'width': 105, 'height': 105}, 'type': 'rectangle'},{'label': 'tongue', 'coordinates': {'x': 383, 'y': 823, 'width': 512, 'height': 491}, 'type': 'rectangle'}]"
/Users/samuelfolledo/Desktop/Convolutional-Neural-Network-Practice/Images/e9f36345-7232-472e-97eb-67e900aa343e.jpg,"[{'label': 'tongue', 'coordinates': {'x': 158, 'y': 1028, 'width': 316, 'height': 944}, 'type': 'rectangle'},{'label': 'tongue', 'coordinates': {'x': 476, 'y': 829, 'width': 542, 'height': 395}, 'type': 'rectangle'},{'label': 'tonsil', 'coordinates': {'x': 739, 'y': 552, 'width': 135, 'height': 166}, 'type': 'rectangle'},{'label': 'tons

# Model Trainer for StrepScan

Train and create an ML model for StrepScan using [Convolutional Neural Network](https://en.wikipedia.org/wiki/Convolutional_neural_network) class

**Make sure we have the following files and directories:**
- **annotations.json** - labeled images
- **Images directory** - folder which contains all the labeled images

In [5]:
import turicreate as tc

## Load Images

In [6]:
images = tc.image_analysis.load_images("Images")
images

path,image
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 500 Width: 375
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 843 Width: 1500
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 360 Width: 480
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 266 Width: 508
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1390 Width: 1300
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1500 Width: 1125
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 180 Width: 320
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 299 Width: 450
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 262 Width: 400
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 321 Width: 400


In [7]:
# images.explore() # show all images

## Load Annotations.csv

In [8]:
annotations = tc.SFrame.read_csv("annotations.csv")
annotations.head()

Finished parsing file /Users/samuelfolledo/Desktop/Convolutional-Neural-Network-Practice/annotations.csv

Parsing completed. Parsed 100 lines in 0.053046 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,list]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/samuelfolledo/Desktop/Convolutional-Neural-Network-Practice/annotations.csv

Parsing completed. Parsed 278 lines in 0.016492 secs.

path,annotations
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'pharynx','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'pharynx','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,"[{'label': 'pharynx','coordinates': {'x': ..."


## Joining SFrames
To be able to train your model, you have to join both SFrames (annotations and images).

In [9]:
joined_sframe = images.join(annotations)
joined_sframe

path,image,annotations
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1500 Width: 843,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1500 Width: 843,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 500 Width: 375,"[{'label': 'pharynx','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 843 Width: 1500,"[{'label': 'pharynx','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 360 Width: 480,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 266 Width: 508,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1390 Width: 1300,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1500 Width: 1125,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 180 Width: 320,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 262 Width: 400,"[{'label': 'pharynx','coordinates': {'x': ..."


## Training
Split dataframe. 80% of the rows/samples will be used for training and the rest will be for testing our model

In [10]:
training_sframe, testing_sframe = joined_sframe.random_split(1.0)
training_sframe

path,image,annotations
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1500 Width: 843,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1500 Width: 843,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 500 Width: 375,"[{'label': 'pharynx','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 843 Width: 1500,"[{'label': 'pharynx','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 360 Width: 480,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 266 Width: 508,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1390 Width: 1300,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 1500 Width: 1125,"[{'label': 'tonsil','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 180 Width: 320,"[{'label': 'tongue','coordinates': {'x': ..."
/Users/samuelfolledo/Desktop/Convolutional-Neu ...,Height: 262 Width: 400,"[{'label': 'pharynx','coordinates': {'x': ..."


In [11]:
testing_sframe

path,image,annotations


## Training SFrame
To train a sample model, we will use the object_detector toolkit and the previously created training_sframe , with 50 iterations

This model will not have high accuracy. **To get better accuracy, you would need to create a model with at least 300 iterations or more**, possibly on more pictures. Note, this can take time

In [12]:
# model_50 = tc.object_detector.create(training_sframe, feature="image", annotations="annotations", max_iterations=50)

In [13]:
# model_50

In [14]:
model_1200 = tc.object_detector.create(training_sframe, feature="image", annotations="annotations", max_iterations=1200)

Download completed: /var/folders/xd/zfjrkl2n7z3bjq6mnbs2w9600000gn/T/model_cache/darknet.params
Download completed: /var/folders/xd/zfjrkl2n7z3bjq6mnbs2w9600000gn/T/model_cache/darknet.mlmodel


Using GPU ("AMD Radeon Pro 450") to create model.

Setting 'batch_size' to 16

+--------------+--------------+--------------+

| Iteration    | Loss         | Elapsed Time |

...

| 1185         | 2.71145      | 17m 55s      |

| 1190         | 2.66085      | 17m 59s      |

| 1195         | 2.56674      | 18m 3s       |

| 1200         | 2.48029      | 18m 7s       |

+--------------+--------------+--------------+

In [15]:
model_1200

Class                                    : ObjectDetector

Schema
------
Model                                    : darknet-yolo
Number of classes                        : 4
Input image shape                        : [3, 416, 416]

Training summary
----------------
Training time                            : 18m 7s
Training epochs                          : 69
Training iterations                      : 1200
Number of examples (images)              : 278
Number of bounding boxes (instances)     : 976
Final loss (specific to model)           : 2.4803

## Export Model

In [18]:
model_1200.export_coreml("throatParts.mlmodel")

In [187]:
metrics_1200 = model_1200.evaluate(testing_sframe)
metrics_1200

{'average_precision_50': {'pharynx': 0.1182892844080925,
  'tongue': 0.6657035946846008,
  'tonsil': 0.34186312556266785,
  'uvula': 0.6363426446914673},
 'mean_average_precision_50': 0.44054967164993286}